In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Titanic CSV Reader") \
    .master("spark://spark:7077") \
    .getOrCreate()

df = spark.read.csv("/data/titanic.csv", header=True, inferSchema=True)

df.show()


+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|gender| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|      

In [3]:
from pyspark.sql.functions import col

df_null_cabin = df.filter(col("Cabin").isNull())

count_by_gender = df_null_cabin.groupBy("gender").count()

count_by_gender.show()



+------+-----+
|gender|count|
+------+-----+
|female|  217|
|  male|  470|
+------+-----+



In [ ]:
from pyspark.sql.functions import mean
average=df.select(mean("Age"))

average.show()

+-----------------+
|         avg(Age)|
+-----------------+
|29.69911764705882|
+-----------------+



In [11]:
average = df.select(mean("Age")).collect()[0][0]

df = df.na.fill({"Age": average})
df.show()


+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|gender|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|             35.0|    0|    0|          373450|

In [ ]:
df.write.option("header", True).mode("overwrite").csv("file:///tmp/depi_folder")

In [21]:
df.groupBy("Survived").count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [22]:
df.groupBy("Embarked").count().orderBy("count", ascending=False).show(5)

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  644|
|       C|  168|
|       Q|   77|
|    NULL|    2|
+--------+-----+



In [23]:
from pyspark.sql.functions import avg

df.groupBy("Pclass").agg(avg("Survived").alias("Survival_Rate")).orderBy("Pclass").show()

+------+-------------------+
|Pclass|      Survival_Rate|
+------+-------------------+
|     1| 0.6296296296296297|
|     2|0.47282608695652173|
|     3|0.24236252545824846|
+------+-------------------+



In [24]:
from pyspark.sql.functions import max, min, avg

df.select(
    max("Fare").alias("Max_Fare"),
    min("Fare").alias("Min_Fare"),
    avg("Fare").alias("Avg_Fare")
).show()


+--------+--------+----------------+
|Max_Fare|Min_Fare|        Avg_Fare|
+--------+--------+----------------+
|512.3292|     0.0|32.2042079685746|
+--------+--------+----------------+



In [25]:
from pyspark.sql.functions import when, col

df_age_grouped = df.withColumn(
    "Age_Group",
    when(col("Age") <= 18, "0-18")
    .when((col("Age") > 18) & (col("Age") <= 35), "19-35")
    .when((col("Age") > 35) & (col("Age") <= 60), "36-60")
    .otherwise("60+")
)

df_age_grouped.groupBy("Age_Group").count().orderBy("Age_Group").show()

+---------+-----+
|Age_Group|count|
+---------+-----+
|     0-18|  139|
|    19-35|  535|
|    36-60|  195|
|      60+|   22|
+---------+-----+

